In [ ]:
# default_exp dataset_builder
%load_ext autoreload
%autoreload 2

In [ ]:
#export 
import random
from rake_nltk import Rake
from kirby.database_proxy import WikiDatabase
import json
import importlib
import spacy
import en_core_web_sm
import tensorflow_hub as hub
from sklearn.neighbors import NearestNeighbors
import json

# Dataset Builder
> Builds a number of types of datasets that are augmented by the `wikidata` knowledge base

## Dataset Variations

Keywords can be supplied to the `build` function through the `dataset_type` keyword argument

### Description

`DatasetBuilder.build(ds, dataset_type='description')`

Augements the given dataset with the description of the `keyword`. 

*Example*

`Stephen Curry is my favorite basketball player. {Stephen Curry: {Description: American basketball player}}`

In [ ]:
#export
class DatasetBuilder():
    "Build a dataset using `get_entities_in_text`"
    def __init__(self):
        self.rake = Rake()
        self.db = WikiDatabase()
        self.nlp = en_core_web_sm.load()
        module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
        self.encoder = hub.load(module_url)
    
    def build(self, ds, dataset_type='random'):
        "Build a database based a given dataset"
        if dataset_type == 'random':
            return ds.map(self.random, batched=False)
        elif dataset_type == 'description':
            return ds.map(self.description, batched=False)
        elif dataset_type == 'relevant':
            pass
        
    def description(self, sequence):
        """Return a description augmented version of the seqeuence `text`"""
        text = sequence['text']
        try:
            keyword_entity = self._keyword_entity(text)
            json_string = self._get_json(keyword_entity)
            # Return concatenated string
            sequence['text'] += " " + json_string
        except Exception as e:
            # We expect there to be an exception when no entities are found
            pass
        return sequence
    
    def _keyword_entity(self, text):
        """Return the entity of the highest ranked keyword"""
        ranked_phrases = self.get_ranked_phrases(text)
        for phrase in ranked_phrases:
            entity = self.db.get_entity_by_label(phrase)
            if entity:
                return entity
        return None
        
    def _get_json(self, item):
        """Return JSON version of list object"""
        d = {"label": None, "description": None}
        d['label'] = item[1]
        d['description'] = item[2]
        return json.dumps(d)
        
    def keyword(self, x):
        ranked_phrases = self.get_ranked_phrases(x)
        return ranked_phrases[0]
    
    def get_ranked_phrases(self, x):
        self.rake.extract_keywords_from_text(x)
        return self.rake.get_ranked_phrases()
    
    #staticmethod
    def add_to_accepted(self, a_sentences, sentence):
        if len(a_sentences) > 2:
            a_sentences.pop(0)
        a_sentences.append(sentence)
    

    def get_entities_in_text(self, text):
        "Returns entities found in the sentence `text`"
        doc = self.nlp(text)
        entities = []
        spacy_entities = doc.ents
        for entity in spacy_entities: 
            entity = self.db.get_entity_by_label(entity.text)
            if entity:
                entities.append(entity)
        return entities  
    
    def get_entity_associations(self, entity_id):
        """
        Given an `entity_id` return a dictionary containing all the associated properties.
        """
        entity_associations_dict = {}
        # Remove all None values from list
        associations = self.db.get_entity_associations(entity_id)
        for property_id, related_entity_id in associations: 
            property_name, related_entity_label = self.db.get_property_string(property_id, related_entity_id)
            entity_associations_dict[property_name] = related_entity_label
        return entity_associations_dict

# Testing

In [ ]:
# Build description dataset
from kirby.run_params import RunParams
from kirby.data_manager import DataManager
from datasets import load_dataset

run_params = RunParams(debug=False)
data_manager = DataManager(run_params)
ds_builder = DatasetBuilder()
block_size = 128

split = 'train'
split = f'{split}[:{run_params.batch_size*block_size if run_params.debug else f"{run_params.data_set_percentage}%"}]'
ds = load_dataset('text', data_files=run_params.data_files, split=split)
aug_train_ds = ds_builder.build(ds, dataset_type='description')

# Save
aug_train_ds.to_csv('data/augmented_datasets/description_train.csv')

split = 'valid'
split = f'{split}[:{run_params.batch_size*block_size if run_params.debug else f"{run_params.data_set_percentage}%"}]'
ds = load_dataset('text', data_files=run_params.data_files, split=split)
aug_valid_ds = ds_builder.build(ds, dataset_type='description')

# Save
aug_valid_ds.to_csv('data/augmented_datasets/description_valid.csv')

INFO:absl:Using /tmp/tfhub_modules to cache modules.


10776

In [ ]:
# creation
ds_builder = DatasetBuilder()
assert isinstance(ds_builder, DatasetBuilder)

In [ ]:
# Test ranked phrases
x = "Stephen Curry is my favorite basketball player."
ds_builder.rake.extract_keywords_from_text(x)
ranked_phrases = ds_builder.rake.get_ranked_phrases()
assert ranked_phrases == ['favorite basketball player', 'stephen curry'], "RAKE failed"

In [ ]:
print(ds_builder.db.get_entity_by_label('Cristiano Ronaldo'))
assert ds_builder.db.get_entity_by_label('Cristiano Ronaldo') == ['Q11571', 'Cristiano Ronaldo', 'Portuguese association football player'], 'ERROR in `database_proxy`'

In [ ]:
# Get Entities from the sentence
x = "Microsoft has bought Bethesda"
entities = ds_builder.get_entities_in_text(x)
print(entities)
assert entities == [['Q2283', 'Microsoft', 'American multinational technology corporation'],\
                    ['Q224892', 'Bethesda', 'Wikimedia disambiguation page']],\
                    'Error in `dataset_builder.get_entities_in_text()`'

In [ ]:
# Get associations from an entity
associations = ds_builder.get_entity_associations(entities[0][0])
assert associations == {"topic's main Wikimedia portal": 'Portal:Microsoft',
 'founded by': 'Bill Gates',
 'country': 'United States of America',
 'instance of': 'software company',
 'headquarters location': 'Redmond',
 'stock exchange': 'NASDAQ',
 'chief executive officer': 'Steve Ballmer',
 "topic's main category": 'Category:Microsoft',
 'subsidiary': 'Xbox Game Studios',
 'described by source': 'Lentapedia (full versions)',
 'industry': 'technology industry',
 'product or material produced': 'Microsoft Windows',
 'legal form': 'Washington corporation',
 'business division': 'Microsoft Research',
 'history of topic': 'history of Microsoft',
 'member of': 'Alliance for Open Media',
 'permanent duplicated item': None,
 'part of': 'NASDAQ-100',
 'award received': 'Big Brother Awards',
 'owner of': 'Microsoft TechNet',
 'owned by': 'BlackRock',
 'board member': 'Reid Hoffman',
 'chairperson': 'John W. Thompson',
 'location of formation': 'Albuquerque',
 'director / manager': 'Satya Nadella',
 'external auditor': 'Deloitte & Touche LLP',
 'partnership with': 'ID2020'}, 'Error in `dataset_builder.get_entity_associations()`'

In [ ]:
# Description
text = "Darth Vader cut off Luke Skywalker's hand"
ds_builder.description(text)